In [1]:
import cv2
import numpy as np

In [2]:
image_path = r'C:\Suraj\projects\openCv\data\projdata\sample\input_image.png'

img = cv2.imread(image_path)
dummy = img.copy()
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# img = cv2.resize(img, (1500,850))
# img_gray = cv2.resize(img_gray, (1500,850))

# cv2.imshow("img", img)
cv2.imshow("gray", img_gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Bilateral Filter

In [3]:
def nothing(x):
    pass

cv2.namedWindow("BilaterFilter")
cv2.createTrackbar("d", "BilaterFilter", 3, 50, nothing)
cv2.createTrackbar("sigmaColor", "BilaterFilter", 0, 500, nothing)
cv2.createTrackbar("sigmaSpace", "BilaterFilter", 0, 500, nothing)

while True:
    d = cv2.getTrackbarPos('d', 'BilaterFilter')
    sigmaColor = cv2.getTrackbarPos('sigmaColor', 'BilaterFilter')
    sigmaSpace = cv2.getTrackbarPos('sigmaSpace', 'BilaterFilter')

    
    bi_f = cv2.bilateralFilter(img_gray, d, sigmaColor, sigmaSpace)
    cv2.imshow("BilaterFilter", bi_f)
    k = cv2.waitKey(1)
    if k == ord("q"):
        break

cv2.destroyAllWindows()

### Threshold Detection

In [4]:
def nothing(x):
    pass

cv2.namedWindow("Threshold")
cv2.createTrackbar("a", "Threshold", 0, 255, nothing)
cv2.createTrackbar("b", "Threshold", 10, 255, nothing)

while True:
    a = cv2.getTrackbarPos('a', 'Threshold')
    b = cv2.getTrackbarPos('b', 'Threshold')
    
    # b = 255
    ret,thresh = cv2.threshold(bi_f,a,b,cv2.THRESH_BINARY_INV)
    cv2.imshow("Threshold", thresh)
    k = cv2.waitKey(1)
    if k == ord("q"):
        break

cv2.destroyAllWindows()

In [5]:
cv2.imshow('Threshold', thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
# canny = cv2.Canny(thresh, 30, 200)

# cv2.imshow('Canny Edge Detection', canny)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

#### Morphological Operation

In [5]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,1))
dilated = cv2.dilate(thresh, kernel, iterations=8)
eroded = cv2.erode(dilated, kernel, iterations=4)

cv2.imshow('Morphological', dilated)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Contours Detection

In [6]:
cnts,hier = cv2.findContours(eroded,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
# cnts,hier = cv2.findContours(eroded,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
# print("Number of contour==",cnts,"\ntotal contour==",len(cnts))
# print("Hierarchy==\n",hier)

In [7]:
img_c = cv2.drawContours(img,cnts,-1,(0,0,255),1)

cv2.imshow("Contour===",img_c)
# cv2.imwrite(r'data/projdata/output_1.png', img)
# cv2.imshow("thresh==",thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
# dummy = img.copy()
# # Loop over the contours
# for i, contour in enumerate(cnts):
#     # Calculate the area of the contour
#     area = cv2.contourArea(contour)

#     # Draw the contour if it has a larger area than the threshold
#     if area < 2000:
#         cv2.drawContours(dummy, cnts, i, (0, 255, 0), -1)  # Draw in green with thickness of 3

# cv2.imshow("Dummy",dummy)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


### Find the preferal of the contours

In [8]:
sum = 0
for c in cnts:
    perimeter = np.round(cv2.arcLength(c,False))
    if perimeter > 10:
        # print(perimeter, '\n')
        sum = sum + perimeter
print(f'Contour Length: {len(cnts)}')
print('Final Length: ',sum)

Contour Length: 409
Final Length:  12147.0


In [15]:
len_inch = sum/120 # 96 is the DPI of the image
len_feet = len_inch*4 # Scale 1 inch is equal to 8 feet 

# 5% error 
per_error = len_feet*0.05

print(f'Pipe length in Inches: {np.round(len_inch,2)}\n')
print(f'approx Pipe length in Feet: {np.round(len_feet, 2)}\n')
print(f"Pipe Length in feet with 5% error probablity\nLower range:{np.round(len_feet-per_error,2)}\nUpper range:{np.round(len_feet+per_error,2)}")

Pipe length in Inches: 100.92

approx Pipe length in Feet: 403.67

Pipe Length in feet with 5% error probablity
Lower range:383.48
Upper range:423.85
